In [14]:
#This Notebook contains a code taht perfectly works for a gif and is a basis to expand in "Brudnopis.ipynb"
#It is not the final version of what is expected

In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import interpolate
from scipy.integrate import ode
import matplotlib.colors as colors
import matplotlib.cbook as cbook
from matplotlib import cm
import animatplot as amp
import imageio


In [2]:
a=5
b= 100
xx = np.linspace(-a, a, b)
yy = np.linspace(-a, a, b)
mX, mY = np.meshgrid(xx,yy)

In [3]:
def stokeslet(f,r0,mX,mY):
    Id=np.array([[1,0],[0,1]])
    r=np.array([mX-r0[0],mY-r0[1]])

    Idf=np.dot(Id,f) 
    
    rTf=(r*f[:,np.newaxis,np.newaxis]).sum(axis=0)
    rrTf=(r*rTf[np.newaxis,])
    modr=(r[0]**2+r[1]**2)**.5
    
    u,v=Idf[:,np.newaxis,np.newaxis]/modr[np.newaxis]+rrTf/modr**3.
    return [u,v]

def B_dir(t,p,fx,fz):
    ex = fx(p[0],p[1])
    ez = fz(p[0],p[1])
    n = (ex**2.0+ez**2.0)**0.5
    return [ex/n, ez/n]

In [4]:
R=0.001
dt=0.8*R
r1=np.array([1.3/np.sqrt(3),1])   # f is direction of the force
f=np.array([0,1])                         # r is position of the force red arrow - touchdown point
r2=np.array([-1.3/np.sqrt(3),1])
r3=np.array([0,0])
x0, x1= -0.99*a, 0.99*a
y0, y1= -0.99*a, 0.99*a

In [5]:
amx = np.linspace(np.cos(np.arctan(2/np.sqrt(3))), np.cos(0), 30)
amy = np.linspace(np.sin(np.arctan(2/np.sqrt(3))), np.sin(0), 30)
amcordinates= np.vstack([amx,amy]).T
appendix= np.array([])
names=[]

In [6]:
#!!!!!!!!!!

#nie klikamy tu

for j in amcordinates:
    fig=plt.figure(figsize=(6,6),facecolor="w")
    ax = plt.axes()
    u1,v1=stokeslet((-.5)*f,np.array([r1[0]*j[0], r1[1]*j[1]]),mX,mY)
    u2,v2=stokeslet((-.5)*f,np.array([r2[0]*j[0], r2[1]*j[1]]),mX,mY)
    u3,v3=stokeslet(f,r3,mX,mY)
    u=u1+u2+u3
    v=v1+v2+v3
    
    # set the starting point of the magnetic field line
    xstart = np.linspace(-0.99*a, 0.99*a, 14)
    additional = np.linspace(-0.99*a, 0.99*a, 8)
    ystart = np.zeros(22)
    for i in additional:
        xstart = np.append(xstart, -i)

    for i in range(0, 22):
        if i<14:
            ystart[i]=0.99*a
        if i>=14:
            ystart[i]=-0.99*a


    places=np.vstack([xstart,ystart]).T

    fbx = interpolate.interp2d(xx,yy,u)
    fbz = interpolate.interp2d(xx,yy,v)

    r=ode(B_dir)
    r.set_integrator('vode')
    r.set_f_params(fbx,fbz)

    xs,ys = [],[]
    for p in places:
        x=[p[0]] 
        y=[p[1]]
        r.set_initial_value([p[0], p[1]], 0)
        while r.successful():
            r.integrate(r.t+dt)
            x.append(r.y[0])
            y.append(r.y[1])
            hit_electrode=False
            if (not (x0<r.y[0] and r.y[0]<x1)) or (not (y0<r.y[1] and r.y[1]<y1)):
                break
        xs.append(x)
        ys.append(y)
    
    Z = np.sqrt(v**2+u**2)
    ax.pcolormesh(mX, mY, Z,
                norm=colors.LogNorm(vmin= 10**(-4), vmax=10**2),
                      snap=True,
               cmap=plt.cm.inferno, rasterized=True, 
               shading='gouraud', zorder=0)
    
    for x,y in zip(xs,ys):
        ax.plot(x,y, color="k" , zorder=10)
        
    plt.savefig('plot'+str(j)+'.png',
                #bbox_inches='tight', pad_inches=0, dpi=400
               )
    
    plt.close(fig='all')


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/scipy/integrate/_ode.py:1013: UserWarning: vode: Excess work done on this call. (Perhaps wrong MF.)
  warnings.warn('{:s}: {:s}'.format(self.__class__.__name__,


In [8]:
import os, sys
import imageio.v2 as imageio

In [9]:
names= []
for j in amcordinates:
    names.append('plot'+str(j)+'.png')


In [10]:
with imageio.get_writer('mygif2.gif', mode='I') as writer:
    for filename in names:
        image = imageio.imread(filename)
        writer.append_data(image)

In [11]:
#os.listdir()
for filename in set(names):
       os.remove(filename)

In [13]:
class TargetFormat(object):
    GIF = ".gif"
    MP4 = ".mp4"
    AVI = ".avi"

def convertFile(inputpath, targetFormat):
    """Reference: http://imageio.readthedocs.io/en/latest/examples.html#convert-a-movie"""
    outputpath = os.path.splitext(inputpath)[0] + targetFormat
    print("converting\r\n\t{0}\r\nto\r\n\t{1}".format(inputpath, outputpath))

    reader = imageio.get_reader(inputpath)
    #fps = reader.get_meta_data()['fps']
    fps=5

    writer = imageio.get_writer(outputpath, fps=fps)
    for i,im in enumerate(reader):
        sys.stdout.write("\rframe {0}".format(i))
        sys.stdout.flush()
        writer.append_data(im)
    print("\r\nFinalizing...")
    writer.close()
    print("Done.")

convertFile("mygif2.gif", TargetFormat.MP4)

converting
	mygif2.gif
to
	mygif2.mp4
frame 29
Finalizing...
Done.


![mygif_gif](mygif.gif)